In [ ]:
# Start writing cod# load init moudle file

$data_import = true
load "/work/pannote/panbot_note.rb"


true

In [ ]:

db_last_transfer_block = Transfer.order(:block_number).last.block_number
last_block = $client.eth_get_block_by_number("latest",false)["result"]["number"].to_i(16)

db_last_block = Block.order(:block_number).last.block_number
db_last_epoch = Epoch.order(:epoch).last.epoch
last_epoch = $contract.call.current_epoch
last_block = $client.eth_get_block_by_number("latest",false)["result"]["number"].to_i(16)

puts "last_epoch = #{last_epoch}"
puts "db_last_epoch = #{db_last_epoch}"
puts "-----------------------"
puts "last_block = #{last_block}"
puts "db_last_block = #{db_last_block}"
puts "-----------------------"
puts "last_transfer_block = #{db_last_transfer_block}"
puts "db_last_block = #{db_last_block}"
puts "-----------------------"

puts "epochs #{last_epoch-db_last_epoch} - blocks #{last_block-db_last_block} - transfer #{last_block-db_last_transfer_block}"
nil

last_epoch = 45139
db_last_epoch = 45124
-----------------------
last_block = 15262548
db_last_block = 15261803
-----------------------
last_transfer_block = 15261199
db_last_block = 15261803
-----------------------
epochs 15 - blocks 745 - transfer 1349


In [ ]:
# Calc Epoch
puts "Epoch"

time = Time.now

epoch_start = Epoch.where(total_count:nil).select(:epoch).map {|x| x.epoch}.min
epoch_end = Epoch.where(total_count:nil).select(:epoch).map {|x| x.epoch}.max
# epoch_start = Epoch.first_epoch
# epoch_end = Epoch.last_epoch
# epoch_start = db_last_epoch+1
# epoch_end = last_epoch - 1

Parallel.map((epoch_start..epoch_end).to_a,in_threads: 10) do |epoch_number|
  puts "time:#{Time.now} epoch_number: #{epoch_number}" if epoch_number%1000==0

  e = Epoch.find_by_epoch(epoch_number)
  next if e.total_count!=nil

  e.total_count = e._event.where("name = ? or name = ?","BetBear","BetBull").count

  start_block = e._block.where("? <= block_time and block_time < ?",e.start_timestamp,e.start_timestamp+30).order(:block_time).first
  e.start_block_number = start_block ? start_block.block_number : 0

  lock_block = e._block.where("? <= block_time and block_time < ?",e.lock_timestamp,e.lock_timestamp+30).order(:block_time).first
  e.lock_block_number = lock_block ? lock_block.block_number : 0
  e.save!

  # block belong to epoch
  local_epoch = Epoch.find_by_epoch(epoch_number)  
  local_block = local_epoch._block
  local_block.in_batches.update_all(epoch: local_epoch.epoch)
  nil
end
nil
puts "calc #{ Time.now-time }"


Epoch
time:2022-02-15 00:15:07 +0000 epoch_number: 43000
time:2022-02-15 00:15:15 +0000 epoch_number: 44000
time:2022-02-15 00:15:24 +0000 epoch_number: 45000
calc 47.961483822


In [ ]:
puts "[epoch null] -- Block #{Block.where("epoch is null").count}  "

[epoch null] -- Block 51859  


In [ ]:
# Calc Epoch-Tx
require 'parallel'
puts "epoch_detail"

time = Time.now

# epoch_start = Epoch.first_epoch
# epoch_end = Epoch.last_epoch
# epoch_start = db_last_epoch+1
# epoch_end = last_epoch - 1


Parallel.map((epoch_start..epoch_end).to_a,in_threads: 10) do |epoch_number|
    puts "time:#{Time.now} epoch_number: #{epoch_number}" if epoch_number%1000==0
    local_epoch = Epoch.find_by_epoch(epoch_number)
    next if local_epoch.detail.size!=0
  
    event_map = {}
    local_epoch.event.each {|event|
        amount = JSON.parse(event.params)["amount"]
        if event.name=="BetBull" or event.name=="BetBear" then
          event_map[event.tx_hash] = [
              event.name,
              amount,
              event.block_number
          ]
        end
    }            
    bull_amount = 0
    bear_amount = 0
    count =0

    local_epoch.block.each {|block|
        tx = event_map.to_a.map {|x| x.flatten}.filter {|x| x[3]==block.block_number}
        tx.each do |x|
          if x[1]=="BetBull" then
              bull_amount=bull_amount+x[2]
          end
          if x[1]=="BetBear" then
              bear_amount=bear_amount+x[2]
          end
        end
        count = count +tx.size
        
        ed = EpochDetail.new()
        ed.block_number = block.block_number
        ed.block_time = block.block_time
        ed.epoch = local_epoch.epoch
        ed.bet_count = count
        ed.bull_amount = bull_amount
        ed.bear_amount = bear_amount
        ed.bull_payout = bull_amount==0?0:(bull_amount+bear_amount)/bull_amount
        ed.bear_payout = bear_amount==0?0:(bull_amount+bear_amount)/bear_amount
        ed.save
            
    }            
end
puts "calc #{ Time.now-time }"
                  


epoch_detail
time:2022-02-15 00:27:40 +0000 epoch_number: 43000
time:2022-02-15 00:27:41 +0000 epoch_number: 44000
time:2022-02-15 00:27:42 +0000 epoch_number: 45000
calc 2.429251007


In [ ]:
Block.where("block_number > ?",15600000).each {|x| x.delete}
Tx.where("block_number > ?",15600000).each {|x| x.delete}
Event.where("block_number > ?",15600000).each {|x| x.delete}
EpochDetail.where("block_number > ?",15600000).each {|x| x.delete}
Epoch.where("epoch >= ?",Epoch.where("lock_block_number > ?",15600000).order(:epoch).first.epoch-1).each {|x| x.delete}


NoMethodError: undefined method `epoch' for nil:NilClass

In [ ]:
puts Block.where("block_number > ?",15600000).count
puts Tx.where("block_number > ?",15600000).count
puts Event.where("block_number > ?",15600000).count
puts EpochDetail.where("block_number > ?",15600000).count
puts Epoch.where("lock_block_number > ?",15600000).count

0
0
0
0
0


In [ ]:
puts Task.where(status:"abort").map {|x| x.status="open"; x.save}

true
true
true
true


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>